#Integration Testing Assignment

In [1]:
!pip install fastapi uvicorn pytest httpx


In [2]:
%%writefile database.py
import sqlite3

DB_NAME = "users.db"

def init_db():
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS users(
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            username TEXT UNIQUE,
            password TEXT
        )
    """)
    conn.commit()
    conn.close()

def add_user(username, password):
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    cursor.execute("INSERT INTO users(username, password) VALUES (?,?)", (username, password))
    conn.commit()
    conn.close()

def get_user(username):
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    cursor.execute("SELECT username, password FROM users WHERE username=?", (username,))
    user = cursor.fetchone()
    conn.close()
    return user


Writing database.py


In [3]:
%%writefile app.py
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import database

database.init_db()

app = FastAPI()

class User(BaseModel):
    username: str
    password: str

@app.post("/register")
def register(user: User):
    try:
        database.add_user(user.username, user.password)
        return {"message": "User registered successfully"}
    except:
        raise HTTPException(status_code=400, detail="User already exists")

@app.post("/login")
def login(user: User):
    db_user = database.get_user(user.username)

    if db_user and db_user[1] == user.password:
        return {"message": "Login successful"}
    raise HTTPException(status_code=401, detail="Invalid username or password")


Writing app.py


In [4]:
%%writefile test_integration.py
from fastapi.testclient import TestClient
from app import app
import database
import os

client = TestClient(app)

def setup_module(module):
    # Reset DB before running tests
    if os.path.exists(database.DB_NAME):
        os.remove(database.DB_NAME)
    database.init_db()

def test_register_user():
    response = client.post("/register", json={"username": "vaishnavi", "password": "1234"})
    assert response.status_code == 200
    assert response.json()["message"] == "User registered successfully"

    # Verify DB entry
    user = database.get_user("vaishnavi")
    assert user[0] == "vaishnavi"
    assert user[1] == "1234"

def test_login_user_success():
    response = client.post("/login", json={"username": "vaishnavi", "password": "1234"})
    assert response.status_code == 200
    assert response.json()["message"] == "Login successful"

def test_login_user_fail():
    response = client.post("/login", json={"username": "vaishnavi", "password": "wrong"})
    assert response.status_code == 401
    assert response.json()["detail"] == "Invalid username or password"


Writing test_integration.py


In [5]:
!pytest -q


...                                                                      [100%]
3 passed in 0.79s
